# import libraries

In [1]:
import numpy as np
import math
import pandas as pd
import cv2 as cv
import copy
from skyfield.api import E, N, W, wgs84, load
from skyfield.data import mpc
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN

# Draw intersect of a line and circle (Given angle from center)

In [2]:
def intersect_circle(angle,r):
#     m = 3.14
    signx = -1
    signy = -1
    quad = 1
    if(angle == 360):
        angle = 0
        
    if(angle>=270):
        signy = 1
        angle = angle - 270
        quad = 4
    elif(angle>=180):
        signx = 1
        signy = 1
        angle = angle - 180
        quad = 3
    elif(angle>=90):
        signx = 1
        angle = angle - 90
        quad = 2
    m=math.tan(math.radians(angle))
#     print(m)
    x = math.sqrt((r*r)/(1+(m*m)))
    y = m * x
    if(quad % 2 ==0):
        x, y = y, x
#     print('x,y',x,y)
    xx = signx * math.floor(x)
    yy = signy * math.floor(y)
    return xx,yy

In [3]:
def intersect_circle_clock(angle,r):
#     m = 3.14
    signx = 1
    signy = -1
    quad = 1
    if(angle == 360):
        angle = 0
        
    if(angle>=270):
        signy = 1
        angle = angle - 270
        quad = 4
    elif(angle>=180):
        signx = -1
        signy = 1
        angle = angle - 180
        quad = 3
    elif(angle>=90):
        signx = -1
        angle = angle - 90
        quad = 2
    m=math.tan(math.radians(angle))
#     print(m)
    x = math.sqrt((r*r)/(1+(m*m)))
    y = m * x
    if(quad % 2 ==0):
        x, y = y, x
#     print('x,y',x,y)
    xx = signx * math.floor(x)
    yy = signy * math.floor(y)
    return xx,yy

# insert image

In [4]:
def insert_image(imgA, p_name, x, y):
    local = 'images/'+p_name+'.jpg'
    obj = cv.imread(local, 0)
    length, bredth = obj.shape
    len_half, bre_half = int(length/2), int(bredth/2)
    p = q = 0
    for i in range(y-len_half, y+len_half):
        q=0
        for j in range(x-bre_half, x+bre_half):
            if(obj[p][q]<240):
                imgA[i][j] = 0
            q=q+1
        p=p+1
    return imgA

# Draw arrow pointing the angle location

In [5]:
def locate_arrow(angle,image,p_name,inner=True):
    
#     p_name = 
    if (inner): # inner (True) means arrow from inside the circle
        r, r_b, r_a = 250, 232, 228
    else:
        r, r_b, r_a = 302, 320, 324
#     x, y = intersect_circle(angle,r)
    x, y = intersect_circle(angle, r) # for arrow head
    # _clock
#     x_base, y_base = intersect_circle(angle,r_b)
    x_base, y_base = intersect_circle(angle, r_b) # for arrow base
    # _clock
    xa, ya = intersect_circle(angle, r_a) #  for text
    # _clock
    img = cv.arrowedLine(image, (x_base+349,349-y_base), (x+349,349-y), 0, 1, tipLength= .5)
#     img = cv.putText(img,p_name[:2],(xa+349,349-ya), font, .3,(0),1,cv.LINE_AA);
#     img = cv.rectangle(img, (xa+349-7,349-ya-7), (xa+349+7,349-ya+7), 0, 1)
    img = insert_image(image, p_name, xa+349, 349-ya)
    return img

# Read angles of planets

In [6]:
def get_angles():
    y = int(input('Year: '))
    m = int(input('month: '))
    d = int(input('day: '))
    h = int(input('hour: '))
    mins = int(input('minute:'))
    sec = int(input('second: '))
    tx = ts.utc(y, m, d, h, mins, sec)
    gg = gangtok.at(tx)
    row = {}
    for i in Planet_names[:-1]:
        q = P[i]
        ra, dec, dist = gg.observe(q).apparent().radec()
        row[i] = ra.hours * 360 / 24
    ra, dec, distance = gg.observe(ceres).radec()
    row['ceres'] = ra.hours *360 / 24
    rowpd = pd.DataFrame([row], columns = Planet_names)
#     print(rowpd)
    return tx, rowpd;

In [7]:
def check_row(degree, name, row):
    a = copy.deepcopy(row)
#     i = list(Planet_names).index(name)
#     r = a.pop(i)
    n = list(Planet_names).index(name) 
    #print(n)
    a = a.drop(name)
    for j in range(n):
        if abs(degree-a[j])<2:
            return False
    return True

## aspect_grid function

In [8]:
def aspect_grid(row, p_names):
    grid = [[0 for i in range(len(row))] for j in range(len(row))]
    for i in range(len(row)):
        for j in range(i):
            s=-1
            diff = abs(row[i] - row[j])
            if(diff>=50 and diff<=70) or (diff>=290 and diff<=310):
                if (diff>70):
                    diff = diff - 240
                s = 60
            elif (diff>=80 and diff<=100) or (diff>=260 and diff<=280):
                if (diff>100):
                    diff = diff - 180
                s = 90
            elif (diff>=110 and diff<=130) or (diff>=230 and diff<=250):
                if (diff>130):
                    diff = diff - 120
                s = 120
            elif (diff>=170 and diff<=190):
                s = 180
            if(s == -1):
                grid[i][j] = -1;
            else:
                a = diff - s
                if(a < 0):
                    b = 'a'
                else: 
                    b = 's'
                cell = [s, b, abs(a)]
                grid[i][j] = cell
    return grid
            

In [9]:
def draw_grid(row, grid, img2):
    x = 80
    y = 80
    for i in range(len(row)):
        for j in range(i):
            if(isinstance(grid[i][j], list)):
                gdata = grid[i][j]
                if(gdata[0] == 90):
                    sname = 'square'
                elif(gdata[0] == 60):
                    sname = 'sextile'
                elif(gdata[0] == 120):
                    sname = 'trine'
                elif(gdata[0] == 180):
                    sname = 'opposition'
#                 sname = 'images/' + sname + '.jpg'
#                 print(sname)
                string = gdata[1].upper()+' '+str(int(round(gdata[2])))
                img2 = insert_image(img2, sname, x+22+(j*45), y-27+(i*45))
                cv.putText(img2,string,(x+12+(j*45), y-12+(i*45)), font, .3,(0),1,cv.LINE_AA);
    return img2

# Calc rising sign

In [10]:
def rising(time):
    horizon = gangtok.at(time).from_altaz(alt_degrees = 0, az_degrees = 180)
    ra, dec, dist = horizon.radec()
    As = ra.hours * 360 / 24
    return As

# Draw houses

In [11]:
def houses(image, t):
    As = rising(t)
    
    for i in range(12):
        thick = 1
        angle = As + (i * 30)
        if(angle >= 360 ):
            angle = angle - 360
        if(i%3 == 0):
            thick = 2
        hangle = angle+15
        if(hangle>=360):
            hangle= hangle - 360
        x1, y1 = intersect_circle(angle,250)
        x, y = intersect_circle(angle,125)
        xh, yh = intersect_circle(hangle,135)
        cv.line(image,(x1+349, 349-y1),(x+349, 349-y),0,thick)
        txt = str(i+1)
        cv.putText(image,txt,(xh+343, 352-yh), font, .3,(0),1,cv.LINE_AA)
        if(i == 0):
            x0, y0 = intersect_circle(angle,270)
            cv.putText(image,'As',(x0+343, 352-y0), font, .5,(0),1,cv.LINE_AA);
    return As, image

# Global variables

In [12]:
image = cv.imread('chart_frame_equal_house.jpg', 0)
image_original=copy.deepcopy(image)
grid_image = cv.imread('images/aspect_grid_frame_withceres.jpg', 0)
grid_original=copy.deepcopy(grid_image)
font = cv.FONT_HERSHEY_SIMPLEX

In [13]:
ts = load.timescale()

In [14]:
planets=[
    'sun',
    'moon',
    'mercury barycenter',
    'venus barycenter',
    'mars barycenter',
    'jupiter barycenter',
    'saturn barycenter',
    'uranus barycenter',
    'neptune barycenter',
    'pluto barycenter'
]
Planet_names = [
    'sun',
    'moon',
    'mercury',
    'venus',
    'mars',
    'jupiter',
    'saturn',
    'uranus',
    'neptune',
    'pluto',
    'ceres'
]
eph = load('de421.bsp')

In [15]:
earth = eph['earth']
gangtok = earth + wgs84.latlon( 27.3314 , 88.6138)
# gg = gangtok.at(time_p)
P= {}
for i, j in zip(planets, Planet_names[:-1]):
    P[j] = eph[i]

In [16]:
with load.open('MPCORB.excerpt.DAT') as f:
    minor_planets = mpc.load_mpcorb_dataframe(f)
row_ceres=minor_planets.iloc[0]
sun = eph['sun']
ceres = sun + mpc.mpcorb_orbit(row_ceres, ts, GM_SUN)

# Main Func

In [17]:
def draw_chart():
    print('Enter Date:-')
#     rown = set_time()
#     if(rown==-1):
#         print('Date out of bound, Pls Enter time b/w 2018-1-1 and 2020-12-31...')
#         return
#     row = calander.loc[rown][1:]
    t, rows = get_angles()
    row = rows.loc[0]
    image = copy.deepcopy(image_original)
    grid_image=copy.deepcopy(grid_original)
    inner_s = True

    for i, j in zip(row, Planet_names) :
        inner_s = check_row(i, j, row)
        image = locate_arrow(i, image, p_name = j, inner = inner_s)
    As, image = houses(image, t)
#     for i,j in zip(row, Planet_names):
#         print(j, ' : ',i , 'deg')
    grid = aspect_grid(row, Planet_names)
#     for i in grid:
#         print(i)
    grid_image = draw_grid(row, grid, grid_image)
    cv.imshow('Natal_Chart', image)
    cv.imshow('Aspect_grid',grid_image)
    
    cv.waitKey(0)
    cv.destroyAllWindows()

In [18]:
draw_chart()

Enter Date:-
Year: 1998
month: 8
day: 11
hour: 16
minute:30
second: 0
